In [1]:
import numpy as np
import cv2
import matplotlib as plt
import scipy.io as sio
import pandas as pd
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import savgol_filter
import soundfile as sf
from scipy import signal

In [3]:
def findeccentricity (ellipse):
    secelip_1=ellipse[1][0]
    secelip_2=ellipse[1][1]
    if secelip_1>secelip_2:
        elip_a=secelip_1
        elip_b=secelip_2
    else:
        elip_a=secelip_2
        elip_b=secelip_1

    elip_c=np.sqrt((elip_a**2)-(elip_b**2))
    eccentricity=elip_c/elip_a
    return eccentricity

def errorcuadraticomedio (matriz1,matriz2):
    errorcuadratico=(np.subtract(matriz1,matriz2))**2
    sumatoria=errorcuadratico.sum()
    alto,ancho=errorcuadratico.shape[:2]
    errormse=sumatoria/(ancho*alto)
    return errormse

def without_subband_mode_intensities(I1):
    M, N = np.shape(I1)
    I2 = np.zeros((M, N), np.double)
    mode1 = np.zeros((1, M), np.double)

    nf = 0
    for nf in range(0, M):
        thisi = I1[nf, :]
        thisi[thisi == np.inf] = np.nan

        maxi = np.max(np.real(thisi[:]))

        mini = np.min(np.real(thisi[:]))

        threshi = np.abs((mini-maxi)/2)

        hvec = np.arange(np.min(np.real(thisi[:])), np.max(np.real(thisi[:])))
        if np.size(hvec) == 1:
            hvec = np.expand_dims(np.linspace(mini, maxi, 2), axis=0)

        histii = np.real(np.histogram(thisi[:], hvec))
        histi = histii[0]

        loc = np.argmax(histi[:])

        mode1_tmp = hvec[loc]
        mode1[0, nf] = mode1_tmp

    # Filtro de promedio Movil
    mode2 = savgol_filter(mode1, 11, 1)
    mode2 = np.transpose(mode2)

    for nf in range(0, M):
        I2[nf, :] = I1[nf, :]-mode2[nf]
    return I2
    

In [ ]:

#fs, senial = wavfile.read(r"D:\\Users\ACER\Desktop\Trabajo Investigacion\Aureas Mono especies\Audios\\JAGUAS259_20121114_144604.wav")
fs, senial = wavfile.read(r"D:\\Users\ACER\Desktop\Trabajo Investigacion\Aureas Mono especies\Audios\\JAGUAS253_20130117_075446.wav")
np.shape(senial.shape[:])[0]
if np.shape(senial.shape[:])[0] <2:
    p,f,t,d=plt.specgram(senial,Fs=fs,NFFT=2048,noverlap=1550)
else:
    p,f,t,d=plt.specgram(senial[:,1],Fs=fs,NFFT=2048,noverlap=1550)

    
intensidad3=p
intensidad_espectrograma=20*(np.log10(np.abs(intensidad3)))
intensidad_filtrogausiano = cv2.GaussianBlur(intensidad_espectrograma, (13,13),sigmaX=2, sigmaY=5)
#auxiliar=pd.DataFrame(intensidad_espectrograma)


In [39]:
#ruta="D:/Users/ACER/Desktop/Trabajo Investigacion/Aureas Mono especies/Audios/JAGUAS253_20130117_075446.wav"
ruta="D:/Users/ACER/Desktop/Trabajo Investigacion/Aureas Mono especies/Audios/JAGUAS259_20121115_110604.wav"


try:
    x, fs = sf.read(ruta)
except RuntimeError:
    print("error en grabacion")


if len(x.shape) == 1:
    senal_audio = x
else:
    x = x.mean(axis=1)
    x = np.squeeze(x)
    senal_audio = x



nmin = round(len(senal_audio) / (60 * fs))
bio_band = (2000, 8000)
tech_band = (200, 1500)
wn = "hann"
size_wn = 1024
nmin = round(len(senal_audio) / (60 * fs))
nperseg = nmin * size_wn
noverlap = 0
nfft = nmin * size_wn


#f, t, p = signal.spectrogram(senal_audio,fs=fs,window=wn,nperseg=nperseg,
#                             noverlap=noverlap,nfft=nfft,detrend="constant",
#                             scaling="density",axis=-1,mode="magnitude")
f,t,p=signal.spectrogram(senal_audio,fs=fs,nfft=2048,nperseg=569,noverlap=0)

intensidad3=p[1:,:]
intensidad_espectrograma=20*(np.log10(np.abs(intensidad3)))
intensidad_filtrogausiano = cv2.GaussianBlur(intensidad_espectrograma, (13,13),sigmaX=2, sigmaY=5)

In [40]:
#suband=sio.loadmat('Witoutsuband_data.mat')
#with_suband=suband['I3']
with_suband=without_subband_mode_intensities(intensidad_filtrogausiano)

with_suband = with_suband * (with_suband >= 0)
#with_suband_normalizada=with_suband/np.max(with_suband)
#blancoNegro=np.round(255*(with_suband_normalizada))
#blancoNegro=blancoNegro.astype(int)
#_,wsub_bin = cv2.threshold(with_suband, 0, 255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
#_,wsub_bin=cv2.adaptiveThreshold(with_suband,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)


In [41]:



cv2.imwrite("Anda2.png",with_suband)
pabinarizar=cv2.imread(r"Anda2.png",0)

#pabinarizar=with_suband

height,width=pabinarizar.shape[:2]
imagen = np.zeros((height,width,3), np.uint8)

#img = cv2.cvtColor(pabinarizar, cv2.COLOR_BGR2GRAY)
_,wsub_bin = cv2.threshold(pabinarizar, 0, 255,type =cv2.THRESH_BINARY+cv2.THRESH_OTSU)
bin_inver=np.flipud(wsub_bin)
#img = cv2.cvtColor(with_suband, cv2.COLOR_RGB2GRAY)
#wsub_bin=cv2.adaptiveThreshold(pabinarizar,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
imagen[:,:,0]=np.flipud(pabinarizar)
imagen[:,:,1]=np.flipud(pabinarizar)
imagen[:,:,2]=np.flipud(pabinarizar)



In [42]:
RectangKernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(9, 7))
opening = cv2.morphologyEx(bin_inver, cv2.MORPH_OPEN, RectangKernel, iterations=1)
CuadKernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(6, 6))
Close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, CuadKernel, iterations=1)


In [47]:
y=cv2.resize(bin_inver,(1033,205),interpolation=cv2.INTER_CUBIC)
y2=cv2.resize(Close,(1033,205),interpolation=cv2.INTER_CUBIC)
cv2.imshow("img",y)
cv2.imshow("img1",y2)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [43]:

contours, hierarchy = cv2.findContours(Close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
a=[]
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    area=cv2.contourArea(cnt)
    exent=area/(w*h)
    try:
        if area>175 and area<40000 and exent>0.2:
            ellipse = cv2.fitEllipse(cnt) #me da los elemetos que componen una elipse
            eccentricity=findeccentricity(ellipse) #uso una funcion que cree para encontrar la exentricidad del elemento
            if eccentricity>0.2:
                a.append(cnt)     
        else:
            continue
    except:
            0
#cv2.ellipse(imagen,((99.5521011352539, 657.787109375),(110.54241180419922, 186.60873413085938),268.20364379882812),(0,255,0),2)
#cv2.ellipse(imagen,ellipse,(0,255,0),2)
#x,y,w,h = cv2.boundingRect(cnt2)
#print(x,y,w,h)


In [ ]:

#rect = cv2.minAreaRect(cnt2)
#box = cv2.boxPoints(rect)
#box = np.int0(box)
#cv2.drawContours(imagen,[box],0,(0,0,255),2)
#tiempo=sio.loadmat("D:\Proyectos Visual Studio\Python-Jupyther\Bioacustica\Variables\Tiempo.mat")
#tiempo=tiempo["T"]
#frecuencia=sio.loadmat("D:\Proyectos Visual Studio\Python-Jupyther\Bioacustica\Variables\Frecuencia.mat")
#frecuencia=frecuencia["F"]
#frecuencia=np.flipud(frecuencia)


In [44]:
for contorno in a:
    x,y,w,h = cv2.boundingRect(contorno)
    cv2.rectangle(imagen,(x,y),(x+w,y+h),(0,255,0),10)
#cv2.drawContours(imagen,a,-1,(0,100,0),5)


In [45]:
tiempo=t
frecuencia=f
tiempo=np.expand_dims(tiempo,axis=0)
frecuencia=np.expand_dims(frecuencia,axis=1)
segmento=[]
segmentos_nor=[]
for element in a:
    timeI,frecma,duration,magfrec=cv2.boundingRect(element)
    posicion=[int(timeI),int(frecma),int(duration),int(magfrec)]
    cv2.rectangle(imagen,(timeI,frecma),(timeI+duration,frecma+magfrec),(0,255,0),10)
    segmento.append([float(tiempo[:,(posicion[0]+1)]),float(tiempo[:,(posicion[0]+posicion[2]-1)]),
    float(frecuencia[(posicion[1]),:]),float(frecuencia[(posicion[1]+posicion[3]-1),:])])
    segmentos_nor.append([posicion[0],posicion[1],posicion[2],posicion[3]])
segm_xie=np.array(segmento)
segmentos_nor=np.array(segmentos_nor)


In [46]:
escalada=cv2.resize(Close,(1033,205),interpolation=cv2.INTER_CUBIC)
muestra=cv2.resize(imagen,(1033,205),interpolation=cv2.INTER_CUBIC)
cv2.imshow("imagen",muestra)
cv2.imshow("imagen2",escalada)
cv2.waitKey(0) 
cv2.destroyAllWindows()


In [ ]:
gausiano=sio.loadmat('gausianblur.mat')
filgau=gausiano["I2"]
openingmat=sio.loadmat('open.mat')
openingmat=openingmat['img_']
closemat=sio.loadmat('cose.mat')
closemat=closemat["img3_"]
errorgausiano=errorcuadraticomedio(filgau,intensidad_filtrogausiano)
erroropening=errorcuadraticomedio(openingmat,opening)
errorclose=errorcuadraticomedio(closemat,Close)

print(erroropening,errorclose,errorgausiano)


In [ ]:
prueba=[]
features=np.array([[-19.58140079, -20.47963333, -17.42474575, -19.02111535],
       [ -1.08292763,  -1.57679018,  -0.78164826,  -0.70270336],
       [ -0.1362791 ,  -0.49108456,  -1.56561794,  -0.69415751],
       [  0.48064605,   0.59210219,   0.3795349 ,   0.05129817]])


In [ ]:
prueba.append([features[:,0]]+[1,2,3,4])
print(prueba)

lista=[1,2,3,4]+[5,6,7,7]

In [ ]:
lista